# Bayesian Parameter Identification
## Setup Model and FEM (according to E2)

In [59]:
# For better printing within jupyter cells (otherwise only the last variable is printed)
import IPython; IPython.core.interactiveshell.InteractiveShell.ast_node_interactivity = "all"

# Standard python libraries
import os

# General computing libraries
import math
import numpy
import scipy.optimize

# Meshing libraries
import gmsh
import meshio

# Finite element libraries
import dolfin as df
df.parameters["allow_extrapolation"] = True

# Plotting libraries
import matplotlib.pyplot as plt

# VTK and visualization
import itkwidgets
import vtk

# Tracking libraries
import dolfin_warp as dwarp

# MEC581 python library
import LIB581

In [60]:
def create_mesh(
        X0,                   # first component of the center of the tube, in mm
        Y0,                   # second component of the center of the tube, in mm
        Ri,                   # internal radius of the tube, in mm
        Re,                   # external radius of the tube, in mm
        l,                    # characteristic size of the mesh cells, in mm
        mesh_folder="Project",     # folder the mesh file
        mesh_basename="mesh", # basename of the mesh file
        verbose=False):       # print or not the GMSH output

    ## Initialization of GMSH
    gmsh.initialize()
    if not (verbose): gmsh.option.setNumber("General.Terminal",0)
    gmsh.clear()

    ## Geometry
    factory = gmsh.model.geo

    # Points
    p0 = factory.addPoint(X0, Y0, 0, l) ### YOUR CODE HERE ###
    # 4 points on the inner loop
    p1 = factory.addPoint(X0+Ri, Y0, 0, l)
    p2 = factory.addPoint(X0, Y0+Ri, 0, l)
    p3 = factory.addPoint(X0-Ri, Y0, 0, l)
    p4 = factory.addPoint(X0, Y0-Ri, 0, l)
    # 4 points on the outer loop
    p5 = factory.addPoint(X0+Re, Y0, 0, l)
    p6 = factory.addPoint(X0, Y0+Re, 0, l)
    p7 = factory.addPoint(X0-Re, Y0, 0, l)
    p8 = factory.addPoint(X0, Y0-Re, 0, l)

    # Curves
    # 4 curves for the inner loop
    l11 = factory.addCircleArc(p1, p0, p2) ### YOUR CODE HERE ###
    l12 = factory.addCircleArc(p2, p0, p3)
    l13 = factory.addCircleArc(p3, p0, p4)
    l14 = factory.addCircleArc(p4, p0, p1)
    # 4 curves for the outer loop
    l21 = factory.addCircleArc(p5, p0, p6)
    l22 = factory.addCircleArc(p6, p0, p7)
    l23 = factory.addCircleArc(p7, p0, p8)
    l24 = factory.addCircleArc(p8, p0, p5)
    

    # Curve loop
    cl = factory.addCurveLoop([l11, l12, l13, l14, l21, l22, l23, l24])### YOUR CODE HERE ###

    # Surface
    s = factory.addPlaneSurface([cl]) ### YOUR CODE HERE ###

    # Synchronization, cf., e.g., https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/python/t1.py
    factory.synchronize()

    # In order to only save nodes and elements of the final surface
    # (i.e., not the construction points like the circle center,
    # nor the line elements of the line entities—remember that 
    # unstructured meshers will first mesh the curves, then the
    # surfaces and the volumes, cf. MEC552 L5.2), we declare it as a
    # "physical" surface.
    ps = gmsh.model.addPhysicalGroup(dim=2, tags=[s])

    ## Mesh
    mesh_gmsh = gmsh.model.mesh

    # Mesh generation
    mesh_gmsh.generate(dim=2)

    # In order to visualize the mesh and perform finite element computation using
    # FEniCS, we need to convert the mesh from the GMSH format to the VTK & FEniCS
    # formats. Since there is no direct converter between these formats, we do
    # that here by writing the mesh to the disc in VTK format using GMSH, which
    # we can then read in various formats later on.
    if not os.path.exists(mesh_folder): os.mkdir(mesh_folder)
    gmsh.write(mesh_folder+"/"+mesh_basename+".vtk")

    # Finalization of GMSH
    gmsh.finalize()

    # To convert the mesh from vtk to dolfin format,
    # we use [meshio](https://github.com/nschloe/meshio).
    mesh_meshio = meshio.read(mesh_folder+"/"+mesh_basename+".vtk")

    # For 2D meshes, we need to remove the third component of points,
    # otherwise FEniCS believes it is 3D…
    mesh_meshio.points = mesh_meshio.points[:, :2]

    # We write the mesh in XDMF format.
    meshio.write(mesh_folder+"/"+mesh_basename+".xdmf", mesh_meshio)

    # Finaly we can read the mesh in FEniCS format.
    mesh = df.Mesh()
    df.XDMFFile(mesh_folder+"/"+mesh_basename+".xdmf").read(mesh)

    return mesh

# Now we test the function.
mesh = create_mesh(X0=0.5, Y0=0.5, Ri=0.2, Re=0.3, l=0.1/5, verbose=1)

# We check some mesh informations.
print ("geometric_dimension:", mesh.geometric_dimension())
print ("num_vertices:", mesh.num_vertices())
print ("num_cells:", mesh.num_cells())

# We can define "measures", which allow to integrate expressions over (part of) the mesh.
dV = df.Measure("dx", domain=mesh) # "dx" means over the domain itself
mesh_V0 = df.assemble(df.Constant(1.) * dV)
print ("mesh_V0", mesh_V0)

# We visualize the mesh, by writing it to disk in VTK format, reading it,
# and using [itkwidgets](https://github.com/InsightSoftwareConsortium/itkwidgets).
LIB581.write_VTU_file("Project/mesh", mesh)
mesh_vtk = LIB581.read_VTU_file("Project/mesh")
itkwidgets.view(geometries=mesh_vtk)

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Starting subloop 1 in curve loop 1 (are you sure about this?)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 20%] Meshing curve 2 (Circle)
Info    : [ 30%] Meshing curve 3 (Circle)
Info    : [ 40%] Meshing curve 4 (Circle)
Info    : [ 50%] Meshing curve 5 (Circle)
Info    : [ 70%] Meshing curve 6 (Circle)
Info    : [ 80%] Meshing curve 7 (Circle)
Info    : [ 90%] Meshing curve 8 (Circle)
Info    : Done meshing 1D (Wall 0.0012497s, CPU 0.000671s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0162339s, CPU 0.016374s)
Info    : 575 nodes 1157 elements
Info    : Writing 'Project/mesh.vtk'...
Info    : Done writing 'Project/mesh.vtk'
geometric_dimension: 2
num_vertices: 574
num_cells: 988
mesh_V0 0.1570795786523808


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [61]:
def create_boundaries(
        X0, Y0, Ri, Re,
        mesh):

    # We create a "MeshFunction",
    # which will allow us to assign an id to each edge of the mesh.

    boundaries_mf = df.MeshFunction(
        value_type="size_t", # size_t is like unisgned int, but more robust with respect to architecture and os
        mesh=mesh,
        dim=1) # 0 for nodes, 1 for edges, 2 for faces, etc.

    # We initialize it to zero.

    boundaries_mf.set_all(0)

    # Now we define geometrical subdomains.

    Si_sd = df.AutoSubDomain(
        lambda x, on_boundary:
            on_boundary and\
            df.near((x[0]-X0)**2+(x[1]-Y0)**2, Ri**2, eps=1e-3))### YOUR CODE HERE ###

    Se_sd = df.AutoSubDomain(
        lambda x, on_boundary:
            on_boundary and\
            df.near((x[0]-X0)**2+(x[1]-Y0)**2, Re**2, eps=1e-3))### YOUR CODE HERE ###

    # And we use the subdomains to mark the different parts of the mesh.

    Si_id = 1; Si_sd.mark(boundaries_mf, Si_id)
    Se_id = 2; Se_sd.mark(boundaries_mf, Se_id)

    return boundaries_mf

# Now we test the function.
X0 = 0.5; Y0 = 0.5; Ri = 0.2; Re = 0.3; l=0.1/3
mesh = create_mesh(X0=X0, Y0=Y0, Ri=Ri, Re=Re, l=l)
boundaries_mf = create_boundaries(X0=X0, Y0=Y0, Ri=Ri, Re=Re, mesh=mesh)

# We can define "measures", which allow to integrate expressions over (part of) the mesh boundary.
dS = df.Measure("ds", domain=mesh, subdomain_data=boundaries_mf) # "ds" means over the domain boundary
mesh_S0 = df.assemble(df.Constant(1) * dS)
print ("mesh_S0", mesh_S0)
Si_id = 1; mesh_Si0 = df.assemble(df.Constant(1) * dS(Si_id))
print ("mesh_Si0", mesh_Si0)
Se_id = 2; mesh_Se0 = df.assemble(df.Constant(1) * dS(Se_id))
print ("mesh_Se0", mesh_Se0)

# We visualize the boundaries.
LIB581.write_VTU_file("Project/boundaries", boundaries_mf)
boundaries_vtk = LIB581.read_VTU_file("Project/boundaries")
itkwidgets.view(geometries=boundaries_vtk)

mesh_S0 3.1394399563914983
mesh_Si0 1.255345531645519
mesh_Se0 1.8840944247459779


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [62]:
# We can define the finite element function space of the model,
# for instance here with second order Lagrange polynomials,

U_fs = df.VectorFunctionSpace(mesh, "Lagrange", degree=2)
U_fs.dim()

# as well as the function into which we will store the solution.
# Dolfin functions are very convenient objects, which contain both
# a symbolic representation of the finite element approximation
# (i.e., the linear combination of the shape functions, which we 
# can derive, integrate, etc.) and the array containing the degrees
# of freedom (all the linear algebra is handled by [PETSc](https://www.mcs.anl.gov/petsc),
# which has a pretty ugly website but is one of the most efficient
# open source libraries for linear algebra, though we can also
# manipulate them as numpy arrays for convenience).

U = df.Function(U_fs, name="U")
U
df.grad(U)
U.vector()
U.vector().get_local()

# We can also define "test" and "trial" functions, which are abstract objects
# used to define linear and bilinear variational forms: a linear form 
# must be linear with respect to the "test" function (which is the U^*
# of the equations); a bilinear form must be linear in both the "test"
# function and "trial" function (which is the U of the equations in the 
# linear setting, the ΔU of the equations in the nonlinear setting, i.e.,
# it is the unknown of the linear problem).

U_test  = df.TestFunction(U_fs)
U_trial = df.TrialFunction(U_fs)

1816

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 9071), VectorElement(FiniteElement('Lagrange', triangle, 2), dim=2)), 9087)

Grad(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 9071), VectorElement(FiniteElement('Lagrange', triangle, 2), dim=2)), 9087))

array([0., 0., 0., ..., 0., 0., 0.])

In [63]:
def create_bcs(
        X0, Y0, Ri, Re, U_fs):

    x1 = [X0+Ri, Y0]
    x1_sd = df.AutoSubDomain(
        lambda x, on_boundary: df.near(x[0], x1[0], eps=1e-3) and df.near(x[1], x1[1], eps=1e-3)) #define the subdomains as the 3 points
    x2 = [X0, Y0+Ri]
    x2_sd = df.AutoSubDomain(
        lambda x, on_boundary: df.near(x[0], x2[0], eps=1e-3) and df.near(x[1], x2[1], eps=1e-3))
    x3 = [X0-Ri, Y0]
    x3_sd = df.AutoSubDomain(
        lambda x, on_boundary: df.near(x[0], x3[0], eps=1e-3) and df.near(x[1], x3[1], eps=1e-3))

    bc1 = df.DirichletBC(U_fs.sub(1), 0, x1_sd, "pointwise") # Block first y translation DoF
    bc2 = df.DirichletBC(U_fs.sub(0), 0, x2_sd, "pointwise") # Block first x translation DoF
    bc3 = df.DirichletBC(U_fs.sub(1), 0, x3_sd, "pointwise") # Block translation DoF through second y translation block
    bcs = [bc1, bc2, bc3]

    return bcs

In [64]:
def create_mesh_boundaries_bcs_solution(
        X0, Y0, Ri, Re, l, degree):

    mesh = create_mesh(X0=X0, Y0=Y0, Ri=Ri, Re=Re, l=l)
    boundaries_mf = create_boundaries(X0=X0, Y0=Y0, Ri=Ri, Re=Re, mesh=mesh)
    U_fs = df.VectorFunctionSpace(mesh, "Lagrange", degree=degree)
    bcs = create_bcs(X0=X0, Y0=Y0, Ri=Ri, Re=Re, U_fs=U_fs)
    U = df.Function(U_fs, name="U")

    return mesh, boundaries_mf, bcs, U

In [65]:
def solve_nonlinear_model(mesh, boundaries_mf, bcs, U, Y, nu, P, init_U=None):

    # Kinematics
    # (We define C as a dolfin.variable,
    #  and derive all quantities (including J!) from it,
    #  such that we can later derive the energy with respect to C.)

    I    = df.Identity(2) ### YOUR CODE HERE ### # Identity
    F    = I + df.grad(U) ### YOUR CODE HERE ### # Deformation gradient
    C    = F.T*F ### YOUR CODE HERE ### # Right Cauchy-Green dilatation tensor
    C    = df.variable(C)
    IC   = df.tr(C) ### YOUR CODE HERE ### # First invariant
    IIIC = df.det(C) ### YOUR CODE HERE ### # Third invariant
    J    = df.sqrt(IIIC) ### YOUR CODE HERE ### # Volume ratio
    E    = 0.5*(C - I) ### YOUR CODE HERE ### # Green-Lagrange strain

    # Virtual work of internal forces
    # (We redefine the parameters as dolfin.Constant,
    #  such that dolfin does not recompile the variational forms
    #  for each value of the parameters.)

    Y     = df.Constant(Y)       # Young modulus (E already used for Green-Lagrange strain)
    nu    = df.Constant(nu)      # Poisson ratio
    lmbda = (Y*nu) / ((1.0 + nu)*(1.0 - 2.0*nu)) ### YOUR CODE HERE ### # Lamé constant (plane strain)
    mu    = Y/(2.0*(1.0 + nu)) ### YOUR CODE HERE ### # Lamé constant

    W  = 0.25*lmbda*(J**2 - 1.0 - 2.0*df.ln(J)) ### YOUR CODE HERE ### # Ogden-Ciarlet-Geymonat bulk energy
    W += 0.5*mu*(IC - 3.0 - 2.0*df.ln(J)) ### YOUR CODE HERE ### # Neo-Hookean energy (plane strain)
    Sigma = df.diff(W, C) ### YOUR CODE HERE ###

    U_test = df.TestFunction(U.function_space())
    delta_E = 0.5*(df.grad(U_test).T*F + F.T*df.grad(U_test)) ### YOUR CODE HERE ###

    dV = df.Measure("dx", domain=mesh)

    Wint = df.inner(Sigma, delta_E)*dV ### YOUR CODE HERE ###

    # Virtual work of external forces
    # (We redefine the parameters as dolfin.Constant,
    #  such that dolfin does not recompile the variational forms
    #  for each value of the parameters.)

    P = df.Constant(P)
    N = df.FacetNormal(mesh)
    dS = df.Measure("ds", domain=mesh, subdomain_data=boundaries_mf)
    Si_id = 1

    Wext = -df.dot(P*N, U_test)*dS(Si_id) ### YOUR CODE HERE ###

    # Nonlinear solver
    # (Note that for nonlinear problems solved using the Newton method,
    #  as is the case here, the initial solution of the iterations has a
    #  strong impact on convergence. Thus, we put an additional optional
    #  parameter to the function: if `init_U=True` (the default), the
    #  solution is initialized at 0 before starting the iterations;
    #  conversely, if `init_U=False`, the solution is not initialized,
    #  so the iterations start from the current displacement fields.

    res = Wint - Wext ### YOUR CODE HERE ###

    U_trial = df.TrialFunction(U.function_space())
    jac = df.derivative(res, U, U_trial) ### YOUR CODE HERE ###
    
    if (init_U): U.vector().zero()

    df.solve(res == 0, U, bcs, J=jac) ### YOUR CODE HERE ###

In [66]:
mesh, boundaries_mf, bcs, U = create_mesh_boundaries_bcs_solution(X0=0.5, Y0=0.5, Ri=0.2, Re=0.3, l=0.1/5, degree=2)
solve_nonlinear_model(mesh=mesh, boundaries_mf=boundaries_mf, bcs=bcs, U=U, Y=10., nu=0.3, P=+1.)

# To visualize the solution, we save it into the VTK format.
# (The displacement field is applied onto the mesh, which then represents the deformed configuration;
#  you can change the multiplicative factor of the displacement field to better see the deformation.)
LIB581.write_VTU_file("Project/model", U)
LIB581.Viewer(meshes="Project/model.vtu").view()

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.323e-01 (tol = 1.000e-10) r (rel) = 1.131e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 3.449e-03 (tol = 1.000e-10) r (rel) = 2.947e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 3.512e-05 (tol = 1.000e-10) r (rel) = 3.001e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 4.475e-10 (tol = 1.000e-10) r (rel) = 3.825e-09 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 9.069e-15 (tol = 1.000e-10) r (rel) = 7.751e-14 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.


## Bayesian Identification
### Set-Up

In [67]:
# Setup the simulation (geometry, mesh, boundaries, BCs, function space)
X0, Y0, Ri, Re, l = 0.5, 0.5, 0.2, 0.3, 0.1/5
mesh, boundaries_mf, bcs, U = create_mesh_boundaries_bcs_solution(
    X0=X0, Y0=Y0, Ri=Ri, Re=Re, l=l, degree=2)

# Define “true” parameters
true_Y = 10.0
nu = 0.3
P_obs = 1.0

# Solve the forward (nonlinear) model with the true Young modulus
solve_nonlinear_model(mesh=mesh, boundaries_mf=boundaries_mf, bcs=bcs, U=U,
                      Y=true_Y, nu=nu, P=P_obs)

# Save the result as the observed displacement (deep copy the field)
U_obs = df.Function(U.function_space())
U_obs.assign(U)


Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.323e-01 (tol = 1.000e-10) r (rel) = 1.131e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 3.449e-03 (tol = 1.000e-10) r (rel) = 2.947e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 3.512e-05 (tol = 1.000e-10) r (rel) = 3.001e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 4.475e-10 (tol = 1.000e-10) r (rel) = 3.825e-09 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 9.069e-15 (tol = 1.000e-10) r (rel) = 7.751e-14 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.


### Define a distance mesaure

In [68]:
dV = df.Measure("dx", domain=mesh)
def compute_distance(U1, U2):
    """Compute the L2 error between two displacement fields."""
    error_sq = df.assemble(df.inner(U1 - U2, U1 - U2) * dV)
    return np.sqrt(error_sq)

### Setup ABC Rejection Algorithm

In [ ]:
# Prior for the Young modulus: Uniform over [Y_min, Y_max]
Y_min, Y_max = 5.0, 15.0

# Number of trials and acceptance threshold (tune epsilon as needed)
N_trials = 1000
epsilon = 0.01  # adjust this threshold to control the acceptance rate

accepted_Y = []
accepted_distances = []

for i in range(N_trials):
    # Sample Y from the uniform prior
    Y_sample = np.random.uniform(Y_min, Y_max)
    
    # Solve the forward model with the sampled Young modulus
    # Note: use init_U=True to reinitialize the displacement field at each iteration.
    solve_nonlinear_model(mesh=mesh, boundaries_mf=boundaries_mf,
                           bcs=bcs, U=U, Y=Y_sample, nu=nu, P=P_obs, init_U=True)
    
    # Compute the discrepancy between the current simulation and the observed data
    d = compute_distance(U, U_obs)
    
    # Accept the sample if the error is below the threshold epsilon
    if d < epsilon:
        accepted_Y.append(Y_sample)
        accepted_distances.append(d)
        print(f"Accepted {i}: Y_sample = {Y_sample:.3f}, distance = {d:.3f}")
    else:
        print(f"Rejected {i}: Y_sample = {Y_sample:.3f}, distance = {d:.3f}")

print("Number of accepted samples:", len(accepted_Y))


Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.630e-01 (tol = 1.000e-10) r (rel) = 1.393e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 4.291e-03 (tol = 1.000e-10) r (rel) = 3.667e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 6.183e-05 (tol = 1.000e-10) r (rel) = 5.284e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 1.938e-09 (tol = 1.000e-10) r (rel) = 1.656e-08 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 1.109e-14 (tol = 1.000e-10) r (rel) = 9.475e-14 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
Accepted 0: Y_sample = 8.786, distance = 0.009
Solving nonlinear variational problem.Rejected 1: Y_sample = 6.905, distance = 0.031

  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 2.530e-01 (tol = 1.000e-10) r (rel) =

Rejected 11: Y_sample = 6.243, distance = 0.043
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.007e-01 (tol = 1.000e-10) r (rel) = 8.609e-01 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 2.350e-03 (tol = 1.000e-10) r (rel) = 2.009e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 1.358e-05 (tol = 1.000e-10) r (rel) = 1.161e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 4.969e-11 (tol = 1.000e-10) r (rel) = 4.246e-10 (tol = 1.000e-09)
  Newton solver finished in 4 iterations and 4 linear solver iterations.
Rejected 12: Y_sample = 12.000, distance = 0.010
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 9.641e-02 (tol = 1.000e-10) r (rel) = 8.240e-01 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 2.185e-03 (tol = 1.000e-10) r (rel

Accepted 23: Y_sample = 10.091, distance = 0.001
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 3.857e-01 (tol = 1.000e-10) r (rel) = 3.296e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 4.788e-02 (tol = 1.000e-10) r (rel) = 4.092e-01 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 3.694e-04 (tol = 1.000e-10) r (rel) = 3.157e-03 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 1.462e-07 (tol = 1.000e-10) r (rel) = 1.249e-06 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 3.389e-14 (tol = 1.000e-10) r (rel) = 2.896e-13 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
Rejected 24: Y_sample = 5.730, distance = 0.055
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.388e-01 (tol = 1.000e-10) r (rel

Solving nonlinear variational problem.Rejected 36: Y_sample = 12.846, distance = 0.014

  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 9.142e-02 (tol = 1.000e-10) r (rel) = 7.813e-01 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 1.993e-03 (tol = 1.000e-10) r (rel) = 1.704e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 9.191e-06 (tol = 1.000e-10) r (rel) = 7.855e-05 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 2.038e-11 (tol = 1.000e-10) r (rel) = 1.742e-10 (tol = 1.000e-09)
  Newton solver finished in 4 iterations and 4 linear solver iterations.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 8.289e-02 (tol = 1.000e-10) r (rel) = 7.084e-01 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 1.667e-03 (tol = 1.000e-10) r (rel) = 1.425e-02 (tol = 1.000e-09)
  Newton iterati

Rejected 47: Y_sample = 12.695, distance = 0.013
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.233e-01 (tol = 1.000e-10) r (rel) = 1.054e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 3.160e-03 (tol = 1.000e-10) r (rel) = 2.700e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 2.807e-05 (tol = 1.000e-10) r (rel) = 2.399e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 2.635e-10 (tol = 1.000e-10) r (rel) = 2.252e-09 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 8.118e-15 (tol = 1.000e-10) r (rel) = 6.938e-14 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
Accepted 48: Y_sample = 10.471, distance = 0.003
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 2.271e-01 (tol = 1.000e-10) r (re

...

  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 8.541e-02 (tol = 1.000e-10) r (rel) = 7.299e-01 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 1.763e-03 (tol = 1.000e-10) r (rel) = 1.507e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 6.889e-06 (tol = 1.000e-10) r (rel) = 5.888e-05 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 1.055e-11 (tol = 1.000e-10) r (rel) = 9.021e-11 (tol = 1.000e-09)
  Newton solver finished in 4 iterations and 4 linear solver iterations.
Rejected 181: Y_sample = 13.487, distance = 0.016
Solving nonlinear variational problem.
Rejected 182: Y_sample = 8.063, distance = 0.016
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.890e-01 (tol = 1.000e-10) r (rel) = 1.615e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 5.126e-03 (tol = 1.000e-10) r (rel) = 4.381e-02 (tol = 1.000e-09)
  New

Solving nonlinear variational problem.Rejected 193: Y_sample = 7.986, distance = 0.017

  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.922e-01 (tol = 1.000e-10) r (rel) = 1.643e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 5.267e-03 (tol = 1.000e-10) r (rel) = 4.502e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 9.186e-05 (tol = 1.000e-10) r (rel) = 7.851e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 5.993e-09 (tol = 1.000e-10) r (rel) = 5.122e-08 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 1.219e-14 (tol = 1.000e-10) r (rel) = 1.042e-13 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
Solving nonlinear variational problem.
Rejected 194: Y_sample = 7.625, distance = 0.021  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 2.090e-01 (tol = 1.000e-10) r (rel

Accepted 205: Y_sample = 11.049, distance = 0.006Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.137e-01 (tol = 1.000e-10) r (rel) = 9.717e-01 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 2.829e-03 (tol = 1.000e-10) r (rel) = 2.418e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 2.130e-05 (tol = 1.000e-10) r (rel) = 1.821e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 1.391e-10 (tol = 1.000e-10) r (rel) = 1.189e-09 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 7.536e-15 (tol = 1.000e-10) r (rel) = 6.441e-14 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.

Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 2.328e-01 (tol = 1.000e-10) r (rel) = 1.989e+00 (tol = 1.000e-09)
  Newton iterat

Accepted 217: Y_sample = 8.842, distance = 0.009
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 1.907e-01 (tol = 1.000e-10) r (rel) = 1.630e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 5.200e-03 (tol = 1.000e-10) r (rel) = 4.444e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 9.008e-05 (tol = 1.000e-10) r (rel) = 7.699e-04 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 5.689e-09 (tol = 1.000e-10) r (rel) = 4.862e-08 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 1.173e-14 (tol = 1.000e-10) r (rel) = 1.003e-13 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
Rejected 218: Y_sample = 8.022, distance = 0.017
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 9.873e-02 (tol = 1.000e-10) r (re

Rejected 229: Y_sample = 13.558, distance = 0.016
Solving nonlinear variational problem.
Rejected 230: Y_sample = 7.479, distance = 0.023
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 2.166e-01 (tol = 1.000e-10) r (rel) = 1.852e+00 (tol = 1.000e-09)
  Newton iteration 2: r (abs) = 6.834e-03 (tol = 1.000e-10) r (rel) = 5.841e-02 (tol = 1.000e-09)
  Newton iteration 3: r (abs) = 1.276e-04 (tol = 1.000e-10) r (rel) = 1.091e-03 (tol = 1.000e-09)
  Newton iteration 4: r (abs) = 1.259e-08 (tol = 1.000e-10) r (rel) = 1.076e-07 (tol = 1.000e-09)
  Newton iteration 5: r (abs) = 1.171e-14 (tol = 1.000e-10) r (rel) = 1.001e-13 (tol = 1.000e-09)
  Newton solver finished in 5 iterations and 5 linear solver iterations.
Solving nonlinear variational problem.
Rejected 231: Y_sample = 5.857, distance = 0.052
  Newton iteration 0: r (abs) = 1.170e-01 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iterat

### Analyse the approximate Posterior

In [ ]:
# Plot the histogram of accepted Young modulus values
plt.hist(accepted_Y, bins=20, density=True, color="skyblue", edgecolor="black")
plt.xlabel("Young's modulus, Y")
plt.ylabel("Approximate posterior density")
plt.title("ABC Posterior Distribution for Young's Modulus")
plt.show()